### Разработка Скрипта

Рассмотрть проект скрипта для распознования и выравнивания изображения в случае поворта на 90-180 градусов.
1. Импорт библиотек:
from google.colab import files
import cv2
import numpy as np
Здесь происходит импорт необходимых библиотек: files из Google Colab для загрузки файлов, cv2 для работы с изображениями через OpenCV, и numpy для работы с массивами.

2. Определение констант:
MAX_FEATURES = 500
GOOD_MATCH_PERCENT = 0.15
Здесь определены константы, которые будут использоваться в процессе выравнивания изображений.

3. Определение функции для выравнивания изображений:
def align_images(im1, im2):
    # ... (см. ниже)
    return im1_reg
Здесь определена функция align_images, которая принимает два изображения (im1 и im2) и возвращает выровненное изображение im1_reg.

4. Основная часть кода:
uploaded = files.upload()

ref_filename = list(uploaded.keys())[0]
im_filename = list(uploaded.keys())[1]

im_reference = cv2.imread(ref_filename, cv2.IMREAD_COLOR)
im = cv2.imread(im_filename, cv2.IMREAD_COLOR)

im_reg = align_images(im, im_reference)

out_filename = "/content/aligned.png"
cv2.imwrite(out_filename, im_reg)

print("Сохраненное выровненное изображение: ", out_filename)

С помощью files.upload() пользователь может загрузить два изображения: ref_filename (ссылка) и im_filename (изображение для выравнивания).
cv2.imread используется для чтения изображений в цветовом формате.
Вызывается функция align_images для выравнивания изображений.
Выровненное изображение сохраняется и выводится сообщение о сохранении.


Чтение изображений : Сначала мы читаем эталонное изображение (или изображение шаблона) и изображение, которое мы хотим выровнять по этому шаблону .
Обнаружение объектов: Затем мы обнаруживаем объекты ORB на двух изображениях. Несмотря на то, что для вычисления гомографии нам нужно всего 4 признака, обычно на двух изображениях обнаруживаются сотни признаков.

Важно при загрузке сачало загружаем первый файл эталон изображения(он будет сравниваться) со вторым файлом который нужно выровнять.

в файле matches.png при сравнении сопостовляется 500 точек (можно задать и меньше) для нахождения точек совпадения с файлом изображением эталом, чтобы его повернуть при нахождении и совпадении точек.

файл aligned.png это обработанное и выравненное изображение которое соответствует файлу источнику 2.png

Необходимо далее проработать скрипт для работы с pdf файлом, чтобы из него нарезать отдельно png или jpg файлы.


**Важно при загрузке сачало загружаем первый файл эталон изображения(он будет сравниваться) со вторым файлом который нужно выровнять.**

In [ ]:
# Ваше решение
# Подключение google disc для загрузки файлов - эталон и сканируемое изображение
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from google.colab import files
import cv2
import numpy as np

MAX_FEATURES = 500
GOOD_MATCH_PERCENT = 0.15

def alignImages(im1, im2):
    # Convert images to grayscale
    im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
    im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)

    # Detect ORB features and compute descriptors.
    orb = cv2.ORB_create(MAX_FEATURES)
    keypoints1, descriptors1 = orb.detectAndCompute(im1Gray, None)
    keypoints2, descriptors2 = orb.detectAndCompute(im2Gray, None)

    # Match features.
    matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = matcher.match(descriptors1, descriptors2)

    # Sort matches by score
    matches = sorted(matches, key=lambda x: x.distance)

    # Remove not so good matches
    numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
    matches = matches[:numGoodMatches]

    # Draw top matches
    imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches, None)
    cv2.imwrite("matches.jpg", imMatches)

    # Extract location of good matches
    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)

    for i, match in enumerate(matches):
        points1[i, :] = keypoints1[match.queryIdx].pt
        points2[i, :] = keypoints2[match.trainIdx].pt

    # Find homography
    h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)

    # Use homography
    height, width, channels = im2.shape
    im1Reg = cv2.warpPerspective(im1, h, (width, height))

    return im1Reg, h

# Загрузка изображений в среду выполнения Colab
print("Первым загрузите эталон изображения.")
print("Втрым загрузите сканируемое изображения.")
uploaded = files.upload()
print("Должно быть загружено как минимум два изображения.")

# Проверка, что загружено как минимум два изображения
if len(uploaded.keys()) < 2:
    print("Ошибка: Должно быть загружено как минимум два изображения.")
else:
    # Пример использования
    imReference = cv2.imread(list(uploaded.keys())[0], cv2.IMREAD_COLOR)
    im = cv2.imread(list(uploaded.keys())[1], cv2.IMREAD_COLOR)

    # Проверка, удалось ли загрузить изображения
    if imReference is None or im is None:
        print("Ошибка: Не удается загрузить изображения.")
    else:
        print("Выравнивание изображений ...")
        # Registered image will be restored in imReg.
        # The estimated homography will be stored in h.
        imReg, h = alignImages(im, imReference)

        # Write aligned image to disk.
        outFilename = "aligned_image.png"
        print("Сохранение выровненного изображения :", outFilename);
        cv2.imwrite(outFilename, imReg)

        # Print estimated homography


Первым загрузите эталон изображения.
Втроым загрузитесканируемое изображения.


Saving 2et.png to 2et.png
Saving img_19.png to img_19.png
Должно быть загружено как минимум два изображения.
Выравнивание изображений ...
Сохранение выровненного изображения : aligned_image.png
